# Sobre o estudo de caso analisado

>**Dados:** O conjunto de dados disponibilizado contém informações sobre funcionários contratado e desligados de uma empresa
>
>**Objetivo:** Analisar os dados para identificar padrões e tendências que expliquem o aumento dos custos com desligamentos e novas contratações, buscando formas de reduzir esses custos.<br>
>
>**Motivação:** A empresa tem dificuldade em manter e desenvolver talentos, possui uma rotatividade grande de colaboradores que são contratados e em pouco tempo são desligados, provocando um custo alto com R&S e com processos demissionais.  

### Importando os pacotes que serão utilizado


In [1]:
# manipulação de dados e cálculos matemáticos
import pandas as pd
import numpy as np
# visualização de dados
import matplotlib.pyplot as plt
import seaborn as sns
# criar análise exploratória rápida dos dados
from ydata_profiling import ProfileReport 

# para ignorar alertas
import warnings
warnings.filterwarnings('ignore')

### Leitura e carregamento dos dados

In [2]:
dados = r'C:\Users\manunes\personal-projects\analise-rh\dataset\Base_dados - Rotatividade RH.xlsx'
df = pd.read_excel(dados, engine='openpyxl')

Os dados de RH desta empresa foram disponibilizados em um arquivo Excel chamado `Base_dados - Rotatividade RH.xlsx`.

O conjunto de dados contém esses campos:

Variáveis            | Descrição
---------------------|--------------
`Data_Contratacao`     | Data exata em que o colaborador foi contratado pela empresa    
`Data_Desligamento`    | Data em que foi desligado, se estiver vazio é porque o colaborador ainda está ativo  
`Meses_de_Servico`     | Tempo total em meses que o colaborador trabalha ou trabalhou na empresa          
`Idade`                | Idade do colaborador no momento da contratação           
`Pontuacao_Desempenho` | Avaliação do desempenho em um intervalo de 1 a 5, onde 5 inidica uma entrega de excelência         
`Satisfacao_Trabalho`  | Indica o nível de satisfação com o trabalho entre 1 a 5, onde 5 significa muito satisfeito        
`Nivel_Cargo`          | Nível hierárquico do cargo ocupado          
`Salario`              | Salário bruto mensal do colaborador        
`Horas_Extras`         | Quantidade de horas extras que o colaborador possui         
`Desligamento`         | Indica o status do colaborador, onde 1 significa que foi desligado e 0 que o colaborador está ativo

## 1. Visão Geral sobre o Dataset

Tamanho do dataset

In [3]:
print('Tamanho do dataset:', df.shape[0], 'Linhas e', df.shape[1], 'Colunas')

Tamanho do dataset: 650 Linhas e 10 Colunas


Cabeçalho e rodapé do dataset

In [4]:
df

,Data_Contratacao,Data_Desligamento,Meses_de_Servico,Idade,Pontuacao_Desempenho,Satisfacao_Trabalho,Nivel_Cargo,Salario,Horas_Extras,Desligamento
0,2024-02-14,NaT,3,24,2.066788,2.043644,Pleno,4994.41,15.465241,0
1,2023-11-16,2024-05-14,6,43,3.256257,1.711719,Sênior,8798.34,19.036627,1
2,2023-11-16,NaT,6,45,4.827732,3.480214,Sênior,8891.02,13.326408,0
3,2023-05-20,NaT,12,31,1.453944,1.617558,Pleno,7606.22,14.209663,0
4,2022-07-24,NaT,22,37,2.088917,2.763390,Sênior,9236.03,11.059687,0
...,...,...,...,...,...,...,...,...,...,...
645,2021-08-28,NaT,33,31,2.260088,1.539328,Pleno,7940.88,6.847854,0
646,2022-02-24,NaT,27,23,4.611915,2.521799,Junior,4893.95,4.583937,0
647,2022-10-22,NaT,19,21,4.688376,4.641638,Junior,3838.70,11.306088,0
648,2021-09-27,NaT,32,32,3.551361,3.998147,Pleno,4739.81,4.343341,0


In [18]:
# Por praticidade, os nomes das variáveis serão convertidos em minúsculos
df.columns = df.columns.str.lower()

Tipos de dados

In [6]:
print('Tipos de dados das variáveis:')
df.dtypes

Tipos de dados das variáveis:


data_contratacao        datetime64[ns]
data_desligamento       datetime64[ns]
meses_de_servico                 int64
idade                            int64
pontuacao_desempenho           float64
satisfacao_trabalho            float64
nivel_cargo                     object
salario                        float64
horas_extras                   float64
desligamento                     int64
dtype: object

## 2. Entendendo os Tipos de Variáveis e Dados (Categorização)

<h3 style="color: orange;">Categorizando os Tipos de Variáveis e de Dados</h3> 

***
#### **Variáveis Numéricas**
<span style="color: gray;">**Dados Quantitativos:**</span>
- **meses_de_servico** - Discretos
- **idade** - Discretos
- **pontuacao_desempenho** - Contínuos
- **satisfacao_trabalho** - Contínuos
- **salario** - Contínuos
- **horas_extras** - Contínuos

<span style="color: gray;">**Dados Temporais:**</span>
- **data_contratacao** - Discretos
- **data_desligamento** - Discretos

<br> 

***

#### **Variáveis Categóricas**
<span style="color: gray;">**Dados Qualitativos:**</span>
- **nivel_cargo** - Ordinais
- **desligamento (status de desligamento)** - Nominal

## 3. Data Profiling | EDA inicial

Gerar o relatório de perfil de dados com ydata-profiling

In [ ]:
perfil_dos_dados = ProfileReport(df, title='Report RH', explorative=True)

perfil_dos_dados.to_notebook_iframe()

## 4. Limpeza e Tratamento dos Dados

#### I - Dados duplicados

In [7]:
df.nunique()

data_contratacao         33
data_desligamento         1
meses_de_servico         33
idade                    32
pontuacao_desempenho    650
satisfacao_trabalho     650
nivel_cargo               3
salario                 650
horas_extras            540
desligamento              2
dtype: int64

Avaliando aqueles que fazem sentido existirem ou não:

´data_desligamento´: Parece estranho temos uma proposta baseada em muitas demissões e todas elas apontarem para apenas um único registro, ou seja, um único dia em que houve demissões. Vamos trazer um detalhamento sobre isso na análise univariada desta variável;

´salario´: Temos um salário diferente para cada funcionário, considerando que o total de contratados é de 650. Isso indica algo preocupante, que é a empresa não ter um padrão de salários entre seus colaboradores, o que pode ter uma influência no nível de satisfação deles. Diferenças salarias são comentadas entre os próprios colaboradores e costumam causar uma má repercurssão;

´nivel_cargo´: Aqui há um indicativo que reforça o quão delicada é a situação de não ter uma padrão nos salários por cargo, visto que só temos 3 níveis de cargo possíveis e 650 salários diferentes distribuídos entre 650 funcionários. Na análise bivariada é interessante observar o comportamento de oscilação de salário em função do nível de cargo ou do tempo de serviço.

#### II - Dados Ausentes

In [8]:
nulos = df.isnull().sum()
nulos

data_contratacao          0
data_desligamento       506
meses_de_servico          0
idade                     0
pontuacao_desempenho      0
satisfacao_trabalho       0
nivel_cargo               0
salario                   0
horas_extras              0
desligamento              0
dtype: int64

In [9]:
print(f'Temos {nulos.sum()} valores ausentes no conjunto de dados.')

Temos 506 valores ausentes no conjunto de dados.


Calculando a porcentagem de valores ausentes em cada variável

In [10]:
porcentagem_nulos = (nulos / len(df) * 100).round(2).sort_values(ascending=False)

print('Variável              | nulos (%)')
porcentagem_nulos

Variável              | nulos (%)


data_desligamento       77.85
data_contratacao         0.00
meses_de_servico         0.00
idade                    0.00
pontuacao_desempenho     0.00
satisfacao_trabalho      0.00
nivel_cargo              0.00
salario                  0.00
horas_extras             0.00
desligamento             0.00
dtype: float64

## 5. Análise Exploratória dos Dados | EDA Detalhada

### Agrupando as variáveis por tipos

#### Variáveis Numéricas

In [ ]:
# Variáveis Qualitativas
qualitativas = df[['nivel_cargo', 'desligamento']]
qualitativas.head(3)

#### Variáveis Temporais

In [ ]:
# Variáveis Quantitativas Temporais
temporais = df[['data_contratacao', 'data_desligamento']]
temporais.head(3)

#### Variáveis Categóricas

In [ ]:
# Variáveis Quantitativas
variaveis_a_desconsiderar = pd.concat([qualitativas, temporais])
quantitativas = df.loc[:, ~df.columns.isin(variaveis_a_desconsiderar)]

quantitativas.head(3)